In [1]:
import pandas as pd
data = pd.read_csv('output.csv')
data

,name,link
0,Harry Potter and the Deathly Hallows (Harry Po...,/book/show/136251.Harry_Potter_and_the_Deathly...
1,"The Hunger Games (The Hunger Games, #1)",/book/show/2767052-the-hunger-games
2,The Kite Runner,/book/show/77203.The_Kite_Runner
3,The Book Thief,/book/show/19063.The_Book_Thief
4,Harry Potter and the Half-Blood Prince (Harry ...,/book/show/1.Harry_Potter_and_the_Half_Blood_P...
...,...,...
195,The Bookseller of Kabul,/book/show/9838.The_Bookseller_of_Kabul
196,If I Stay,/book/show/4374400-if-i-stay
197,Dear John,/book/show/5526.Dear_John
198,"Sex, Drugs, and Cocoa Puffs: A Low Culture Man...",/book/show/599.Sex_Drugs_and_Cocoa_Puffs


In [2]:
for i in range(len(data['link'])):
    link = "https://www.goodreads.com" + data['link'][i]
    data['link'][i] = link
#data[data['link'] == '/book/show/1.Harry_Potter_and_the_Half_Blood_Prince']
(data[data['link'] == 'https://www.goodreads.com/book/show/1.Harry_Potter_and_the_Half_Blood_Prince'])

,name,link
4,Harry Potter and the Half-Blood Prince (Harry ...,https://www.goodreads.com/book/show/1.Harry_Po...


In [3]:
from scrapy.crawler import CrawlerProcess
from scrapy import Spider
class MySpider(Spider):
    name = 'book'
    allowed_domains = ['https://www.goodreads.com']
    start_urls = list(data['link'])[0:10]

    def parse(self, response):
        table = response.xpath('//*[@id="metacol"]')
        link = response.request.url
        yield{
        'title' : (data[data['link'] == link])['name'].values[0],  
        'author': table.xpath('*[@id="bookAuthors"]/span[2]/div/a/span//text()').extract(),
        'book_link': response.request.url,
        'genre_and_vote' : table.xpath('/html/body/div[2]/div[3]/div[1]/div[2]/div[3]/div[6]/div/div[2]/div')
        }

process = CrawlerProcess(settings={
    "FEEDS": {
        "test.csv": {"format": "csv"},
    },
})
process.crawl(MySpider)
process.start()

2021-11-02 12:22:28 [scrapy.utils.log] INFO: Scrapy 2.5.0 started (bot: scrapybot)
2021-11-02 12:22:28 [scrapy.utils.log] INFO: Versions: lxml 4.6.3.0, libxml2 2.9.5, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.7.0, Python 3.9.7 (tags/v3.9.7:1016ef3, Aug 30 2021, 20:19:38) [MSC v.1929 64 bit (AMD64)], pyOpenSSL 20.0.1 (OpenSSL 1.1.1l  24 Aug 2021), cryptography 3.4.8, Platform Windows-10-10.0.22471-SP0
2021-11-02 12:22:28 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-11-02 12:22:28 [scrapy.crawler] INFO: Overridden settings:
{}
2021-11-02 12:22:28 [scrapy.extensions.telnet] INFO: Telnet Password: 0156a3117f0f493a
2021-11-02 12:22:28 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2021-11-02 12:22:28 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrap

In [31]:
result = pd.read_csv('test.csv')
for title in result['title']:
    title = str(title).replace('      ','')
    print(str(title).replace('\n',''))

The Da Vinci Code
Harry Potter and the Order of the Phoenix
Harry Potter and the Prisoner of Azkaban
Harry Potter and the Goblet of Fire
Harry Potter and the Chamber of Secrets
Harry Potter and the Sorcerer's Stone
Twilight
Memoirs of a Geisha
nan
nan
